COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name:
#### Date:

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure. 

As usual, the first code cell simply imports all the modules we'll be using...

In [7]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [14]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

# Sample usage:  get the list of NEOs for a given week, then lookup the latest NEO record in that list.
week_start = '2023-01-15'
neos = get_neos(week_start)
print(f'{len(neos)} Near Earth Objects found for week of {week_start}')
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

118 Near Earth Objects found for week of 2023-01-15
{'absolute_magnitude_h': 24.86,
 'close_approach_data': [{'close_approach_date': '1949-07-04',
                          'close_approach_date_full': '1949-Jul-04 22:19',
                          'epoch_date_close_approach': -646710060000,
                          'miss_distance': {'astronomical': '0.419726307',
                                            'kilometers': '62790161.51016609',
                                            'lunar': '163.273533423',
                                            'miles': '39015997.166588442'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '73509.6739172087',
                                                'kilometers_per_second': '20.4193538659',
                                                'miles_per_hour': '45676.0526626122'}},
                         {'close_approach_date': '1950-08-07',
                        

## Exercise 1: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body. 
* Develop a class named `CloseApproach` to represent a *single* close approach record. 
* State variables are
    * orbiting body (`str`)
    * approach date (`datetime` object!)
    * miss distance (`float` in km, document it!)
    * relative velocity (`float` in km/hr, ditto)

* Operations must include:
    * `__init__(self, ...)` method to initialize a new object with specific data values
    * `__str__(self)` method to return a nicely formatted string representation of the object.

Write a little code to test your new class.

In [9]:
# Ex. 1 your code here

## Exercise 2: Factory method: CloseApproach.from_dict

We want to be able to construct CloseApproach objects easily from a data record returned from the NEO API.  

Add an "object factory" method to your class...   

    @classmethod
    def from_dict(cls, record):
        ... 

This is a `classmethod` because it provides an alternate way to create and initialize a `CloseApproach` instance.  It takes a dictionary for a single `close_approach_data` record, constructs and returns a `CloseApproach` object representing that same record.
This kind of method is called a “Factory” because it handles the details of constructing an object from raw materials.

Add the `classmethod` to the class you defined in Ex. 1 above.
Remember to convert each element from the data dictionary to the correct type (e.g., parse the date/time string into a `datetime` object).
Add little code to test your new factory method in the code block above.

## Exercise 3:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables representing a single NEO.  Your Asteroid class should define at least 4 "state variables:”
* id  (`int`)
* name (`str`)
* estimated_diameter (`float` in m)
* is_potentially_hazardous (`bool`)

Operations must include:
* `__init__(self, ...)` method to initialize a new object with specific data values
* `__str__(self)` method to return a nicely formatted string representation of the object.
* `from_neo(cls, neo_id)` a "factory" `classmethod` to construct an Asteroid object from the NEO API.  

```
@classmethod
def from_neo(cls, neo_id):
    ... 
```

This "factory" takes the `id` for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.  *Hint*: I provided the code fetch a NEO from its `id` above.

Write a little code to test your new class.

In [10]:
# Ex. 3 your code here

## Exercise 4: Add list of CloseApproach objects to the Asteroid

Every `Asteroid` should have a list of “close approaches”.
Add an instance variable to your Asteroid class `__init__` with a default value of an empty list:

    ...
    self.close_approaches = []
    ...
      
In the `from_neo` factory method, use a list comprehension to build the list of `CloseApproach` objects for the `Asteroid` instance:
```
    self.close_approaches = [...]
```  
Setting the value of an object's instance variables from outside the class is generally frowned upon - this is why we make the factory a method of the class itself!

Now add a new method to `Asteroid` to return the `CloseApproach` object from the asteroid representing the nearest to Earth:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [11]:
# Ex. 4 your test code for nearest_miss here

## Challenge - Take your skills to the next level...
### Exercise 5: develop a useful analysis / data product

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  nearest_miss
    * apply Python’s min function to identify the Asteroid with the nearest_miss

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, `estimated_mass`, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the `CloseApproach` class, `impact_force`,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [12]:
# Ex. 5 (challenge) your code here